In [1]:
import pandas as pd
import os
import openai
from dotenv import dotenv_values

In [2]:
openai.api_key = dotenv_values("../.env")["openai_api"]

In [3]:
df = pd.read_csv("../data/raw/ChinaTweets_Aug2021_Jan2023_01162023.csv")
df["text"].head()

0    By January 23, 2020, Wuhan was placed under qu...
1    Anytime the radical left makes excuses for Chi...
2    During Senate hearings, I often ask witnesses ...
3    Stop blaming unvaccinated Americans.\n\nStart ...
4    Russia has a well established TRIAD: submarine...
Name: text, dtype: object

In [5]:
prompt = "Determine if the following Tweet from a US politician is relevant towards China or not. Answer only as True, if relevant, and False otherwise: \n"
prompt += df["text"].iloc[12]

openai.Completion.create(engine = "text-davinci-003", prompt = prompt)["choices"][0]["text"].strip()

'False.'

In [4]:
def isRelevant(tweet):
    
    prompt = "Determine if the following Tweet from a US politician is relevant towards China or not. Answer only as True, if relevant, and False otherwise: \n"
    prompt += tweet
    
    response = openai.Completion.create(engine = "text-davinci-003", prompt = prompt)
    answer = response["choices"][0]["text"].strip()
    
    return answer

In [7]:
df["gpt_relevance"] = df["text"].apply(isRelevant)